In [ ]:
# 29 Mar 2023
# nrobot

In [ ]:
# 0. Dataset description
# -- Graph # of cases vs month over last 10 years

# POSSIBLE Case studies:
# (See if anything interesting comes up...)

# 1. How was use of 'undercover' changed over time
# -- graph by every month for 10 years
# -- in both absolute counts and % counts of total cases
# -- Probably need to avoid coutning multiple times by using the case id?

# 2. How have fines changed over time
# -- (Don't divide by type of offense to start)

# Would be really cool to see impulse change around the robert kraft case (or
# other media news) since that was in florida

# -- If time: Could be more interesting to extract type of offense and then see
# A. how the ratio has changed over time (NOTE to self: like the extreme poverty split by
# region of world graph)
# B. how the amount of the fines depends on the type of offense or other
# covariates 

# So many possibilities!!!! Could also auto assign ethnicities by last name,
# etc. 
# Or pull out stats about the sex workers using date of birth etc.

In [ ]:
# double check which python version running on which computer 
!which python
!uname -n

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pathlib
#import scipy
import calplot

In [ ]:
import altair as alt

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
# - Load raw data: 
cases = pd.read_csv('nogit_data/case_summaries.csv', index_col=0)

# -- put text as column in csv 
files = list( pathlib.Path('nogit_data/text').glob('*.txt'))

corpus = {}
for file in files:
    with open(file, 'r') as f:
        corpus[file.stem] = ''.join(f.readlines())
df_text = pd.DataFrame.from_dict(corpus, orient='index').reset_index()#.columns=['pdf_text']
df_text.columns = ['doc_id', 'pdf_text']

# -- preproces
cases.date = pd.to_datetime(cases['date'])

# -- drop duplicate cases (as PDFs will likely include pages of other pdf)
# keep oldest case by setting date as the index
cases.set_index('date', inplace=True)
cases.drop_duplicates(subset='case', keep='first', inplace=True)
cases.reset_index(inplace=True)

TEST_CASE = 201104833
assert cases[cases.case == TEST_CASE].date.dt.year.values == [2012] # should be 2012, not 2013

annot_text = pd.merge(left=cases, right=df_text, left_on='doc_id', right_on='doc_id')
annot_text.fillna(value={'pdf_text':''}, inplace=True)

# - Create features:
annot_text['undercover'] = annot_text['pdf_text'].apply(lambda x: 'undercover' in x)

# - Plot features:
#print(annot_text.columns)
#print(annot_text.dtypes)

annot_text.set_index('date', inplace=True)
annot_text[(annot_text.index > '1994') & (annot_text.index < '2023')].resample('Y').count()
_data = annot_text[(annot_text.index > '1990') & (annot_text.index < '2023')].resample('Y').count().reset_index()
# -- ?!?! NOTE: altair bar chart is off by one, so 
# -- manually shift the data  when plotting
_data['_year'] = _data.date - pd.DateOffset(years=1) 

In [ ]:
_data

In [ ]:
annot_text.columns


alt.Chart(annot_text).transform_aggregate(
    count = 'count(doc_id)',
    groupby=['date']).mark_bar().encode(
        x = alt.X('year(date):O'),
        y = alt.Y('count:Q'),
   #     color = 'undercover'
    ) 

In [ ]:
annot_text.reset_index(inplace=True)
_tmp = annot_text[['undercover', 'doc_id']].groupby([annot_text.date.dt.year, 'undercover']).aggregate('count')

In [ ]:
_tmp2 = _tmp.reset_index()
_tmp2 = _tmp2[(_tmp2.date > 1995 ) & (_tmp2.date < 2023)]

In [ ]:
base = alt.Chart(_tmp2, title='# of Cases Per Year in Florida',
     ).encode(
        x=alt.X('date:O', title='Year'),
        y=alt.Y('doc_id', title='# of Cases'),
     )
bars = base.mark_bar(
        size=20, opacity=0.8
    ).encode(
        color='undercover', # CANNOT include or else text labels are calculated in wrong place
)
text = base.mark_text(
    align='center',
    dy=-15,
    size=15
).encode(
    text='doc_id',
    color=alt.Color('undercover'),
    detail = 'undercover',
    #color=alt.Color('undercover',scale=alt.Scale(range=['gray', 'lightgray']), legend=None)
)


#(bars + text + bars.mark_point(size=1000, opacity=0.03, tooltip=alt.TooltipContent("data"))).properties(width=30*30).configure_axis(
(bars + text).properties(width=30*30).configure_axis( # ORDER MATTERS FOR ADDITION for opacity!
    labelFontSize=20,
    titleFontSize=20
).configure_title(
    fontSize=25
)
#).configure_mark(opacity=0.8).configure(autosize='fit')


In [ ]:
new_bars = alt.Chart(_tmp2, title='# of Cases Per Year in Florida', 
    ).mark_bar(
        size=20, opacity=0.6,
    ).encode(
        x=alt.X('date:O', title='Year'),
        y=alt.Y('doc_id', title='# of Cases'),
    )

text = new_bars.mark_text(
    align='center',
    baseline='line-top',
    dy=-15,
    fontSize=15,
).encode(text='doc_id', color=alt.Color('undercover', 
    scale=alt.Scale(scheme="dark2") ))

(new_bars + text + bars.mark_point(size=1000, opacity=0.03, tooltip=alt.TooltipContent("data"))).properties(width=30*30).configure_axis(
    labelFontSize=20,
    titleFontSize=20
).configure_title(
    fontSize=25
)
